### pCrunch Update

Jake Nunemaker

Last Updated: 08/27/2023 (gbarter)

In [1]:
import os
from fnmatch import fnmatch

import numpy as np
import pandas as pd
import ruamel.yaml as ry

from pCrunch import LoadsAnalysis, PowerProduction, FatigueParams
from pCrunch.io import load_FAST_out
from pCrunch.utility import save_yaml, get_windspeeds, convert_summary_stats

def valid_extension(fp):
    return any([fnmatch(fp, ext) for ext in ["*.outb", "*.out"]])

#### Project Directory

In [13]:
output_dir = os.path.join( os.path.dirname(os.path.realpath('')), 
                                           "pCrunch", "test","io","data")
results_dir = os.path.join(output_dir, "results")
save_results = True

outfiles = [
    os.path.join(output_dir, f) for f in os.listdir(output_dir)
    if valid_extension(f)
]
outfiles.sort()
print(f"Found {len(outfiles)} files.")

Found 9 files.


#### Interacting with output files

In [14]:
# The new framework provides an object oriented framework to interact with
# output files. The easiest way to use this is to use the 'load_FAST_out' function.

outputs = load_FAST_out(outfiles[:3])
print([type(m) for m in outputs])

[<class 'pCrunch.io.openfast.OpenFASTAscii'>, <class 'pCrunch.io.openfast.OpenFASTBinary'>, <class 'pCrunch.io.openfast.OpenFASTAscii'>]


In [15]:
# An instance of 'OpenFASTBinary' (or 'OpenFASTAscii' if applicable) is created.
# The instance stores the raw data but also provides many useful methods for
# interacting with the data:

# print(outputs[0].data)
# print(outputs[0].time)
print(outputs[0].channels)
# print(outputs[0].maxima)
# print(outputs[0].stddevs)

# Individual channel time series can also be accessed with dict style indexing:
#outputs[0]["Wind1VelX"]

['Time' 'Wind1VelX' 'Wind1VelY' 'Wind1VelZ' 'TipDxb3' 'TipDyb3' 'TipRDxb3'
 'TipRDyb3' 'Spn5ALxb1' 'Spn5ALyb1' 'RotSpeed' 'LSSGagV' 'HSShftV'
 'RootFxb3' 'RootFyb3' 'RootMEdg3' 'RootMFlp3' 'Spn4MLxb1' 'Spn4MLyb1'
 'LSSGagFxs' 'LSSGagFys' 'LSSGagFzs' 'LSShftTq' 'HSShftTq' 'LSShftPwr'
 'HSShftPwr' 'GenTq' 'GenPwr']


#### pCrunch Configuration

In [16]:
# Channel magnitudes are defined in a dict:
magnitude_channels = {
    "RootMc1": ["RootMxc1", "RootMyc1", "RootMzc1"],
    "RootMc2": ["RootMxc2", "RootMyc2", "RootMzc2"],
    "RootMc3": ["RootMxc3", "RootMyc3", "RootMzc3"],
}

# Define channels (and their fatigue slopes) in a dict:
fatigue_channels = {
    "RootMc1": FatigueParams(lifetime=25.0, slope=10.0, ult_stress=6e8),
    "RootMc2": FatigueParams(lifetime=25.0, slope=10.0, ult_stress=6e8),
    "RootMc3": FatigueParams(lifetime=25.0, slope=10.0, ult_stress=6e8),
}

# Define channels to save extreme data in a list:
channel_extremes = [
    "RotSpeed",
    "RotThrust",
    "RotTorq",
    "RootMc1",
    "RootMc2",
    "RootMc3",
]

#### Run pCrunch

In [17]:
# The API has changed and is in more of an object oriented framework.
la = LoadsAnalysis(
    outfiles[:5],                           # The primary input is a list of output files
    magnitude_channels=magnitude_channels,  # All of the following inputs are optional
    fatigue_channels=fatigue_channels,      # 
    extreme_channels=channel_extremes,      #
    trim_data=(0,),                         # If 'trim_data' is passed, all input files will
)                                           # be trimmed to (tmin, tmax(optional))

la.process_outputs(cores=1,                 # Once LoadsAnalysis is configured, process outputs with
                   return_damage=True,      # optional return of Palmgren-Miner damange and
                   goodman=True)            # optional use of goodman correction for mean load values
                                            # Note `cores` is optional but will trigger parallel processing if configured

#### Outputs

In [18]:
# The summary stats per each file are here:
la.summary_stats

,Wind1VelX,Wind1VelY,Wind1VelZ,TipDxb3,TipDyb3,TipRDxb3,TipRDyb3,Spn5ALxb1,Spn5ALyb1,RotSpeed,...,Anch7Ten,Anch7Ang,Fair8Ten,Fair8Ang,Anch8Ten,Anch8Ang,TipSpdRat,RotCp,RotCt,RotCq
,min,min,min,min,min,min,min,min,min,min,...,integrated,integrated,integrated,integrated,integrated,integrated,integrated,integrated,integrated,integrated
AOC_WSt.out,12.0,-0.0,0.0,-0.06095,-0.008998,-0.146800,-1.280000,-0.003632,-13.630000,1.016000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AOC_WSt.outb,12.0,-0.0,0.0,-0.06095,-0.008998,-0.146838,-1.280184,-0.003632,-13.630744,1.015954,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DLC2.3_1.out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.049300,...,0.0,0.0,12876.9250,4834.6250,0.0,0.0,270.594943,11.814832,31.005534,1.591906
DLC2.3_2.out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.049710,...,0.0,0.0,12910.2725,4830.3545,0.0,0.0,272.806077,12.052701,31.765697,1.613663
DLC2.3_3.out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.053140,...,0.0,0.0,12897.7175,4832.5860,0.0,0.0,272.238298,11.956121,31.413593,1.604839


In [19]:
# These are indexable by channel, stat:
la.summary_stats["RootMc1"]

,min,max,std,mean,median,abs,integrated
AOC_WSt.out,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AOC_WSt.outb,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DLC2.3_1.out,459.805830,9134.167593,2672.055750,5018.940468,6147.974788,9134.167593,301137.859314
DLC2.3_2.out,277.648587,9079.452302,2643.592784,5134.911236,6378.012638,9079.452302,308109.805751
DLC2.3_3.out,347.604352,8986.223847,2657.649371,5099.936575,6449.890708,8986.223847,306007.537691


In [20]:
la.summary_stats[("RootMc1", 'min')]

AOC_WSt.out            NaN
AOC_WSt.outb           NaN
DLC2.3_1.out    459.805830
DLC2.3_2.out    277.648587
DLC2.3_3.out    347.604352
Name: (RootMc1, min), dtype: float64

In [21]:
# Or by file
la.summary_stats.loc["DLC2.3_1.out"]

Wind1VelX  min                  NaN
Wind1VelY  min                  NaN
Wind1VelZ  min                  NaN
TipDxb3    min                  NaN
TipDyb3    min                  NaN
                            ...    
Anch8Ang   integrated      0.000000
TipSpdRat  integrated    270.594943
RotCp      integrated     11.814832
RotCt      integrated     31.005534
RotCq      integrated      1.591906
Name: DLC2.3_1.out, Length: 1113, dtype: float64

In [11]:
# Damage equivalent loads are found here:
la.DELs

,RootMc1,RootMc2,RootMc3
Test1.outb,5372.081676,5487.951352,5267.372170
step_0.outb,28836.838406,16210.632541,33824.200074
Test2.outb,6372.006628,6484.887751,6548.485829
DLC2.3_3.out,5708.372415,5607.452586,2132.788161
AOC_WSt.outb,NaN,NaN,NaN


In [12]:
# Palmgren-Miner damage can be viewed with:
la.damage

,RootMc1,RootMc2,RootMc3
Test1.outb,2.610118e-42,3.230998e-42,2.143750e-42
step_0.outb,5.184544e-35,1.633931e-37,2.555698e-34
Test2.outb,1.438786e-41,1.714979e-41,1.890789e-41
DLC2.3_3.out,4.790238e-42,4.007654e-42,2.539284e-46
AOC_WSt.outb,NaN,NaN,NaN


In [13]:
# Extreme events:
la.extreme_events

{'RotSpeed': [{'Time': 273.4000031799078,
   'RotSpeed': 11.55297565460205,
   'RotThrust': 745.5087280273438,
   'RotTorq': 3122.724365234375,
   'RootMc1': 9590.92475660036,
   'RootMc2': 8392.58661304791,
   'RootMc3': 9467.578392491985},
  {'Time': 8.0,
   'RotSpeed': 7.361658573150635,
   'RotThrust': 2297.14599609375,
   'RotTorq': 13777.115234375,
   'RootMc1': 41062.48033129582,
   'RootMc2': 49675.9908595564,
   'RootMc3': 48690.10941389519},
  {'Time': 463.1000060066581,
   'RotSpeed': 14.304065704345703,
   'RotThrust': 725.6522827148438,
   'RotTorq': 4229.85791015625,
   'RootMc1': 7988.654888226473,
   'RootMc2': 7507.5203383009475,
   'RootMc3': 9389.899303100416},
  {'Time': 61.9,
   'RotSpeed': 11.33,
   'RotThrust': 317.0,
   'RotTorq': 140.8,
   'RootMc1': 2092.704239399347,
   'RootMc2': 3457.296428656357,
   'RootMc3': 3844.6280631681398},
  {'Time': 35.0,
   'RotSpeed': 109.06758293648133,
   'RootMc1': nan,
   'RootMc2': nan,
   'RootMc3': nan}],
 'RotThrust': [{